# In this jupyter notebook, we predict the KM values for one or multiple genome scale models:

In [1]:
from downloading_BiGG_models import *
from directory_infomation import *
from KM_prediction import *
#from functions_and_dicts_data_preprocessing_GNN import *
import os

## 1. Creating input DataFrames:

#### (a) Choosing the BiGG models for which we want to make predictions:

In [2]:
model_IDs = ["iML1515"]
len(model_IDs)

1

#### (b) Getting all amino acid sequences for all enzymes:

In [3]:
def map_sequences_to_df_Km(df_KM, df_sequences, model_ID):
    Uniprot_df = pd.read_csv(join(datasets_dir, "BiGG_GSM", model_ID,
                                  "Uniprot_Mapping_" + model_ID + ".csv"), sep = ";")
    
    df_KM["Sequence"] = np.nan

    for ind in df_KM.index:
        UID = df_KM["Uniprot ID"][ind]
        if not pd.isnull(UID):
            seq = list(Uniprot_df["Sequence"].loc[Uniprot_df["Uniprot ID"] == UID])[0]
            df_KM["Sequence"][ind] = seq
            df_sequences = df_sequences.append({"Uniprot ID": UID, "Sequence" : seq}, ignore_index=True)
    
    df_KM.to_pickle(join(datasets_dir, "BiGG_GSM", model_ID, "df_KM_checkpoint_" + model_ID + ".pkl"))
    return(df_sequences)

In [4]:
df_sequences = pd.DataFrame(columns = ["Uniprot ID", "Sequence"])

for model_ID in model_IDs:
    df_KM = pd.read_pickle(join(datasets_dir, "BiGG_GSM", model_ID, "df_KM_checkpoint_" + model_ID + ".pkl"))
    df_sequences = map_sequences_to_df_Km(df_KM, df_sequences, model_ID)


df_sequences.to_pickle(join(datasets_dir, "BiGG_data", "df_sequences.pkl"))
df_sequences.drop_duplicates()

Uniprot ID                                           Sequence
0        P0A8F4  MTDQSHQCVIIGIAGASASGKSLIASTLYRELREQVGDEHIGVIPE...
4        P0A9M5  MSEKYIVTWDMLQIHARKLASRLMPSEQWKGIIAVSRGGLVPGALL...
12       P0A9M2  MKHTVEVMIPEAEIKARIAELGRQITERYKDSGSDMVLVGLLRGSF...
16       P69441  MRIILLGAPGAGKGTQAQFIMEKYGIPQISTGDMLRAAVKSGSELG...
20       P0A763  MAIERTFSIIKPNAVAKNVIGNIFARFEAAGFKIVGTKMLHLTVEQ...
...         ...                                                ...
6664     P69902  MSTPLQGIKVLDFTGVQSGPSCTQMLAWFGADVIKIERPGVGDVTR...
6668     P76081  MTTFHSLTVAKVESETRDAVTITFAVPQPLQEAYRFRPGQHLTLKA...
6672     P76083  MMINVQTVAVIGSGTMGAGIAEVAASHGHQVLLYDISAEALTRAID...
6676     P75893  MNIVDQQTFRDAMSCMGAAVNIITTDGPAGRAGFTASAVCSVTDTP...
6684     P30748  MIKVLFFAQVRELVGTDATEVAADFPTVEALRQHMAAQSDRWALAL...

[915 rows x 2 columns]

#### (c) Creating a DataFrame with all enzyme-substrate pairs:

In [5]:
df_KM_all_models = pd.DataFrame(columns = ['BiGG reaction ID', 'reaction name', 'gene_reaction_rule',
                                            'substrate', 'Uniprot ID', 'KEGG ID', 'bigg.metabolite', 'metanetx ID',
                                            'substrate name', 'SMILES', 'Sequence', 'model_ID'])

for model_ID in model_IDs:
    df_KM = pd.read_pickle(join(datasets_dir, "BiGG_GSM", model_ID, "df_KM_checkpoint_" + model_ID + ".pkl"))
    df_KM["model_ID"] = model_ID
    df_KM_all_models = pd.concat([df_KM_all_models, df_KM], ignore_index= True)
df_KM_all_models

BiGG reaction ID                               reaction name  \
0              CYTDK2                       Cytidine kinase (GTP)   
1              CYTDK2                       Cytidine kinase (GTP)   
2              CYTDK2                       Cytidine kinase (GTP)   
3              CYTDK2                       Cytidine kinase (GTP)   
4                XPPT          Xanthine phosphoribosyltransferase   
...               ...                                         ...   
7709           OCTNLL  Octanoate non-lipoylated apo domain ligase   
7710           OCTNLL  Octanoate non-lipoylated apo domain ligase   
7711           OCTNLL  Octanoate non-lipoylated apo domain ligase   
7712           OCTNLL  Octanoate non-lipoylated apo domain ligase   
7713           OCTNLL  Octanoate non-lipoylated apo domain ligase   

     gene_reaction_rule substrate Uniprot ID KEGG ID bigg.metabolite  \
0                 b2066    cytd_c     P0A8F4  C00475            cytd   
1                 b2066     gtp_c     P0A8F4  C00044             gtp   
2                 b2066     cmp_c     P0A8F4  C00055             cmp   
3                 b2066     gdp_c     P0A8F4  C00035             gdp   
4                 b0238    prpp_c     P0A9M5  C00119            prpp   
...                 ...       ...        ...     ...             ...   
7709              b4386     atp_c     P32099  C00002             atp   
7710              b4386    octa_c     P32099  C06423            octa   
7711              b4386     amp_c     P32099  C00020             amp   
7712              b4386  octapb_c     P32099  C06423          octapb   
7713              b4386     ppi_c     P32099  C00013             ppi   

     metanetx ID                          substrate name SMILES  \
0        MNXM338                                Cytidine    NaN   
1         MNXM51                       GTP C10H12N5O14P3    NaN   
2         MNXM31                          CMP C9H12N3O8P    NaN   
3         MNXM30                       GDP C10H12N5O11P2    NaN   
4         MNXM91  5-Phospho-alpha-D-ribose 1-diphosphate    NaN   
...          ...                                     ...    ...   
7709       MNXM3                       ATP C10H12N5O13P3    NaN   
7710     MNXM750                      Octanoate (n-C8:0)    NaN   
7711      MNXM14                         AMP C10H12N5O7P    NaN   
7712  MNXM147531               Octanoate (protein bound)    NaN   
7713      MNXM11                             Diphosphate    NaN   

                                               Sequence model_ID  
0     MTDQSHQCVIIGIAGASASGKSLIASTLYRELREQVGDEHIGVIPE...  iML1515  
1     MTDQSHQCVIIGIAGASASGKSLIASTLYRELREQVGDEHIGVIPE...  iML1515  
2     MTDQSHQCVIIGIAGASASGKSLIASTLYRELREQVGDEHIGVIPE...  iML1515  
3     MTDQSHQCVIIGIAGASASGKSLIASTLYRELREQVGDEHIGVIPE...  iML1515  
4     MSEKYIVTWDMLQIHARKLASRLMPSEQWKGIIAVSRGGLVPGALL...  iML1515  
...                                                 ...      ...  
7709  MSTLRLLISDSYDPWFNLAVEECIFRQMPATQRVLFLWRNADTVVI...  iML1515  
7710  MSTLRLLISDSYDPWFNLAVEECIFRQMPATQRVLFLWRNADTVVI...  iML1515  
7711  MSTLRLLISDSYDPWFNLAVEECIFRQMPATQRVLFLWRNADTVVI...  iML1515  
7712  MSTLRLLISDSYDPWFNLAVEECIFRQMPATQRVLFLWRNADTVVI...  iML1515  
7713  MSTLRLLISDSYDPWFNLAVEECIFRQMPATQRVLFLWRNADTVVI...  iML1515  

[7714 rows x 12 columns]

## 2. Making predictions if enzyme and substrate information is available:

#### (a) Creating two lists with all substrates and all enzymes (substrate at position i will be combined with enzyme at position i):

In [6]:
substrates = []
enzymes = []

for ind in df_KM_all_models.index:
    if not pd.isnull(df_KM_all_models["KEGG ID"][ind]):
        substrates.append(df_KM_all_models["KEGG ID"][ind])
    else:
        substrates.append(df_KM_all_models["SMILES"][ind])
        
    enzymes.append(df_KM_all_models["Sequence"][ind])
    
    
for i, sub in enumerate(substrates):
    if pd.isnull(sub):
        substrates[i] = ""
        
for i, enz in enumerate(enzymes):
    if pd.isnull(enz):
        enzymes[i] = ""

#### (b) Making predictions

In [23]:
df = KM_predicton(substrate_list = substrates, 
             enzyme_list = enzymes)

Step 1/3: Calculating numerical representations for all metabolites.
.....1(a) Calculating input matrices for Graph Neural Network
.......Metabolite string 'G00689' could be neither classified as a valid KEGG ID, InChI string or SMILES string
.......Mol file for KEGG ID 'C01834' is not available. Try to enter InChI string or SMILES for the metabolite instead.
.......Mol file for KEGG ID 'C01641' is not available. Try to enter InChI string or SMILES for the metabolite instead.
.......Mol file for KEGG ID 'C07292' is not available. Try to enter InChI string or SMILES for the metabolite instead.
.......Mol file for KEGG ID 'C01496' is not available. Try to enter InChI string or SMILES for the metabolite instead.
.....1(b) Calculating numerical metabolite representations using a Graph Neural Network
Step 2/3: Calculating numerical representations for all enzymes.
.....2(a) Loading ESM-1b model.
.....2(b) Calculating enzyme representations.
Step 3/3: Making predictions for KM.
[15:40:20] WA

#### (c) Find all values for which either substrate or enzyme information was missing:

In [8]:
for ind in df_KM_all_models.index:
    seq, kegg_id = df_KM_all_models["Sequence"][ind], df_KM_all_models["KEGG ID"][ind]
    sub_name, UID = df_KM_all_models["substrate name"][ind], df_KM_all_models["Uniprot ID"][ind]
    try:
        KM = list(df["KM [mM]"].loc[df["enzyme"] == seq].loc[df["substrate"] == kegg_id])[0]
    except IndexError:
        df["KM [mM]"][ind] = np.nan
        
df.loc[pd.isnull(df["KM [mM]"])]

substrate                                             enzyme  \
28      C02637                                                      
29      C00005                                                      
30      C00006                                                      
31      C00493                                                      
268             MYYPFVRKALFQLDPERAHEFTFQQLRRITGTPFEALVRQKVPAKP...   
...        ...                                                ...   
7704                                                                
7705    C00533                                                      
7706                                                                
7707    C14819                                                      
7708    C00887                                                      

                                                GNN rep  \
28    [2.1642337, 56.90529, 2.7549818, 16.883226, 0....   
29    [18.05114, 183.68982, 9.2988615, 95.257034, 0....   
30    [21.702522, 177.2186, 10.161067, 106.1599, 0.9...   
31    [0.6542455, 53.816666, 0.5565872, 17.964241, 0...   
268   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
...                                                 ...   
7704  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
7705  [0.0, 0.549775, 0.27351955, 0.0, 0.0, 0.415074...   
7706  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
7707  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
7708  [0.034241106, 1.0257747, 0.46646684, 0.0, 0.37...   

                                             enzyme rep  complete  KM [mM]  
28    [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...      True      NaN  
29    [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...      True      NaN  
30    [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...      True      NaN  
31    [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...      True      NaN  
268   [0.032019433, 0.18150918, -0.10186252, 0.05456...      True      NaN  
...                                                 ...       ...      ...  
7704  [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...      True      NaN  
7705  [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...      True      NaN  
7706  [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...      True      NaN  
7707  [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...      True      NaN  
7708  [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...      True      NaN  

[1986 rows x 6 columns]

In [9]:
df["prediction method"] = np.nan
df["prediction method"].loc[~pd.isnull(df["KM [mM]"])] = "enzyme and substrate information"

## 3. Making predictions if only substrate information is available:

Loading prediction model:

In [10]:
bst = pickle.load(open(join(datasets_dir, "saved_models", "xgboost", 
                            "xgboost_model_substrate_only_trained_with_all_data.dat"), "rb"))

Making predictions:

In [11]:
for ind in df.index:
    if pd.isnull(df["KM [mM]"][ind]):
        gnn_rep = df["GNN rep"][ind]
        if gnn_rep != "":
            if not all(gnn_rep == np.zeros(52)):
                dX = xgb.DMatrix(np.reshape(gnn_rep, (1,52)))
                df["KM [mM]"][ind] = 10**bst.predict(dX)
                df["prediction method"][ind] = "substrate information"

## 4. Making predictions if only enzyme information is available:

Loading prediction model:

In [12]:
bst = pickle.load(open(join(datasets_dir, "saved_models", "xgboost", 
                            "xgboost_model_esm1b_only_trained_with_all_data.dat"), "rb"))

Making predictions:

In [13]:
for ind in df.index:
    if pd.isnull(df["KM [mM]"][ind]):
        enzyme = df["enzyme"][ind]
        if not pd.isnull(enzyme) and enzyme != "":
            enz_rep = df["enzyme rep"][ind]
            if len(enz_rep) > 1:
                dX = xgb.DMatrix(np.reshape(enz_rep, (1,1280)))
                df["KM [mM]"][ind] = 10**bst.predict(dX)
                df["prediction method"][ind] = "enzyme information"

## 5. Using the mean as a prediction if neither substrate nor enzyme information is available

In [14]:
df["prediction method"].loc[pd.isnull(df["KM [mM]"])] = "mean"
df["KM [mM]"].loc[pd.isnull(df["KM [mM]"])] =  10**-0.767634

## 6. Mapping all predictions to df_KM_all_models:

In [15]:
df_KM_all_models["prediction method"] = np.nan
df_KM_all_models["KM [mM] (predicted)"] = np.nan

for ind in df_KM_all_models.index:
    seq, kegg_id = df_KM_all_models["Sequence"][ind], df_KM_all_models["KEGG ID"][ind]
    sub_name, UID = df_KM_all_models["substrate name"][ind], df_KM_all_models["Uniprot ID"][ind]
    if pd.isnull(seq):
        seq = ""
    if pd.isnull(kegg_id):
        kegg_id = ""
    try:
        KM = list(df["KM [mM]"].loc[df["enzyme"] == seq].loc[df["substrate"] == kegg_id])[0]
        method = list(df["prediction method"].loc[df["enzyme"] == seq].loc[df["substrate"] == kegg_id])[0]
    except IndexError:
        KM = np.nan
        
    if not pd.isnull(KM):
        df_KM_all_models["prediction method"][ind] = method
        df_KM_all_models["KM [mM] (predicted)"][ind] = KM
    else:
        df_KM_all_models["prediction method"][ind] = "mean"
        df_KM_all_models["KM [mM] (predicted)"][ind] = 10**-0.767634
df_KM_all_models

BiGG reaction ID                               reaction name  \
0              CYTDK2                       Cytidine kinase (GTP)   
1              CYTDK2                       Cytidine kinase (GTP)   
2              CYTDK2                       Cytidine kinase (GTP)   
3              CYTDK2                       Cytidine kinase (GTP)   
4                XPPT          Xanthine phosphoribosyltransferase   
...               ...                                         ...   
7709           OCTNLL  Octanoate non-lipoylated apo domain ligase   
7710           OCTNLL  Octanoate non-lipoylated apo domain ligase   
7711           OCTNLL  Octanoate non-lipoylated apo domain ligase   
7712           OCTNLL  Octanoate non-lipoylated apo domain ligase   
7713           OCTNLL  Octanoate non-lipoylated apo domain ligase   

     gene_reaction_rule substrate Uniprot ID KEGG ID bigg.metabolite  \
0                 b2066    cytd_c     P0A8F4  C00475            cytd   
1                 b2066     gtp_c     P0A8F4  C00044             gtp   
2                 b2066     cmp_c     P0A8F4  C00055             cmp   
3                 b2066     gdp_c     P0A8F4  C00035             gdp   
4                 b0238    prpp_c     P0A9M5  C00119            prpp   
...                 ...       ...        ...     ...             ...   
7709              b4386     atp_c     P32099  C00002             atp   
7710              b4386    octa_c     P32099  C06423            octa   
7711              b4386     amp_c     P32099  C00020             amp   
7712              b4386  octapb_c     P32099  C06423          octapb   
7713              b4386     ppi_c     P32099  C00013             ppi   

     metanetx ID                          substrate name SMILES  \
0        MNXM338                                Cytidine    NaN   
1         MNXM51                       GTP C10H12N5O14P3    NaN   
2         MNXM31                          CMP C9H12N3O8P    NaN   
3         MNXM30                       GDP C10H12N5O11P2    NaN   
4         MNXM91  5-Phospho-alpha-D-ribose 1-diphosphate    NaN   
...          ...                                     ...    ...   
7709       MNXM3                       ATP C10H12N5O13P3    NaN   
7710     MNXM750                      Octanoate (n-C8:0)    NaN   
7711      MNXM14                         AMP C10H12N5O7P    NaN   
7712  MNXM147531               Octanoate (protein bound)    NaN   
7713      MNXM11                             Diphosphate    NaN   

                                               Sequence model_ID  \
0     MTDQSHQCVIIGIAGASASGKSLIASTLYRELREQVGDEHIGVIPE...  iML1515   
1     MTDQSHQCVIIGIAGASASGKSLIASTLYRELREQVGDEHIGVIPE...  iML1515   
2     MTDQSHQCVIIGIAGASASGKSLIASTLYRELREQVGDEHIGVIPE...  iML1515   
3     MTDQSHQCVIIGIAGASASGKSLIASTLYRELREQVGDEHIGVIPE...  iML1515   
4     MSEKYIVTWDMLQIHARKLASRLMPSEQWKGIIAVSRGGLVPGALL...  iML1515   
...                                                 ...      ...   
7709  MSTLRLLISDSYDPWFNLAVEECIFRQMPATQRVLFLWRNADTVVI...  iML1515   
7710  MSTLRLLISDSYDPWFNLAVEECIFRQMPATQRVLFLWRNADTVVI...  iML1515   
7711  MSTLRLLISDSYDPWFNLAVEECIFRQMPATQRVLFLWRNADTVVI...  iML1515   
7712  MSTLRLLISDSYDPWFNLAVEECIFRQMPATQRVLFLWRNADTVVI...  iML1515   
7713  MSTLRLLISDSYDPWFNLAVEECIFRQMPATQRVLFLWRNADTVVI...  iML1515   

                     prediction method  KM [mM] (predicted)  
0     enzyme and substrate information             0.189513  
1     enzyme and substrate information             0.086163  
2     enzyme and substrate information             0.098226  
3     enzyme and substrate information             0.096791  
4     enzyme and substrate information             0.082845  
...                                ...                  ...  
7709  enzyme and substrate information             0.027388  
7710  enzyme and substrate information             0.092491  
7711  enzyme and substrate information             0.013122  
7712  enzyme and substrate information             0.092491  
7713  e

## 7. Creating excel table sheet with all models:

In [16]:
df_KM_all_models = pd.DataFrame(data = {"BiGG reaction ID" : df_KM_all_models["BiGG reaction ID"],
                                        "reaction name" : df_KM_all_models["reaction name"],
                                        "BiGG metabolite ID" : df_KM_all_models["bigg.metabolite"],
                                        "substrate name" : df_KM_all_models["substrate name"],
                                        "MetaNetX ID" : df_KM_all_models["metanetx ID"],
                                        "UniProt ID" : df_KM_all_models["Uniprot ID"],
                                        "KM [mM] (predicted)" : df_KM_all_models["KM [mM] (predicted)"],
                                        "prediction method" : df_KM_all_models["prediction method"],
                                        "BiGG model ID" : df_KM_all_models["model_ID"]})

df_KM_all_models.drop_duplicates(inplace = True)

In [18]:
from UliPlot.XLSX import auto_adjust_xlsx_column_width

with pd.ExcelWriter(join(datasets_dir, "BiGG_data", 'KM_GEM_predictions.xlsx')) as writer:
    for model_ID in model_IDs:
        df_KM_model = df_KM_all_models.loc[df_KM_all_models["BiGG model ID"] == model_ID]
        df_KM = df_KM_model.drop(columns = ["BiGG model ID"]).reset_index(drop = True)
        df_KM = df_KM.sort_values(by=['BiGG reaction ID']).reset_index(drop = True)
        df_KM.to_excel(writer, sheet_name = model_ID, index=False)
        auto_adjust_xlsx_column_width(df_KM, writer, sheet_name = model_ID, margin=0)
        
with pd.ExcelWriter(join(datasets_dir, "BiGG_data", 'KM_GEM_predictions.xlsx')) as writer:
    for model_ID in model_IDs:
        df_KM_model = df_KM_all_models.loc[df_KM_all_models["BiGG model ID"] == model_ID]
        df_KM = df_KM_model.drop(columns = ["BiGG model ID"]).reset_index(drop = True)
        df_KM = df_KM.sort_values(by=['BiGG reaction ID']).reset_index(drop = True)
        df_KM.to_excel(writer, sheet_name = model_ID, index=False)
        
        sheetname = model_ID
        worksheet = writer.sheets[sheetname]  # pull worksheet object
        for idx, col in enumerate(df_KM):  # loop through all columns
            series = df_KM[col]
            max_len = max((
                series.astype(str).map(len).max(),  # len of largest item
                len(str(series.name))  # len of column name/header
                )) + 1  # adding a little extra space
            #worksheet.set_column(idx, idx, min(max_len,70))
        
    writer.save()